In [32]:
import clip
import torch
import copy
from torch import nn
from tqdm import tqdm
from typing import Optional
import torch.nn.functional as F
from torch import Tensor

In [3]:
detr_clip = torch.load('outputs/detr_clip_train0.pth')

In [5]:
sample = detr_clip[0]

In [6]:
list(sample.keys())

['detr_f', 'clip_f', 'outputs', 'targets']

In [7]:
sample['detr_f'].size(), sample['clip_f'].size(), list(sample['outputs'].keys()), list(sample['targets'].keys())

(torch.Size([100, 256]),
 torch.Size([1024]),
 ['pred_logits', 'pred_boxes'],
 ['boxes', 'labels', 'image_id', 'area', 'iscrowd', 'orig_size', 'size'])

In [29]:
class DetrClipFuser(nn.Module):
    def __init__(self, d_detr, d_clip, text_encoded, nhead=12) -> None:
        super().__init__()
        self.d_model = d_clip
        self.text_encoded = text_encoded
        self.linear_projection = nn.Linear(d_detr, d_clip)
        decoder_layer = nn.TransformerDecoderLayer(d_model=self.d_model, nhead=nhead)
        self.decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=6)
    
    def forward(self, clip_img_feature, detr_proposals):
        detr_projected = self.linear_projection(detr_proposals)
        print(detr_projected.shape)
        out = self.decoder(tgt=detr_projected, memory=clip_img_feature)
        out = out.transpose(0, 1)
        logits = out @ self.text_encoded.T
        return logits

In [30]:
detr_features = torch.rand(100, 8, 256)
clip_features = torch.rand(1, 8, 1024)
texts_encoded = torch.rand(80, 1024)

detr_clip_fuser = DetrClipFuser(256, 1024, texts_encoded, nhead=8)
out = detr_clip_fuser(clip_features, detr_features)
out.size()

torch.Size([100, 8, 1024])


torch.Size([8, 100, 80])

In [36]:
def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")


def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


class TransformerDecoder(nn.Module):

    def __init__(self, decoder_layer, num_layers, norm=None, return_intermediate=False):
        super().__init__()
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        self.return_intermediate = return_intermediate

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        output = tgt

        intermediate = []

        for layer in self.layers:
            output = layer(output, memory, tgt_mask=tgt_mask,
                           memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask,
                           memory_key_padding_mask=memory_key_padding_mask,
                           pos=pos, query_pos=query_pos)
            if self.return_intermediate:
                intermediate.append(self.norm(output))

        if self.norm is not None:
            output = self.norm(output)
            if self.return_intermediate:
                intermediate.pop()
                intermediate.append(output)

        if self.return_intermediate:
            return torch.stack(intermediate)

        return output.unsqueeze(0)


class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self, tgt, memory,
                     tgt_mask: Optional[Tensor] = None,
                     memory_mask: Optional[Tensor] = None,
                     tgt_key_padding_mask: Optional[Tensor] = None,
                     memory_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None,
                     query_pos: Optional[Tensor] = None):
        
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        q = k = self.with_pos_embed(tgt, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

    def forward_pre(self, tgt, memory,
                    tgt_mask: Optional[Tensor] = None,
                    memory_mask: Optional[Tensor] = None,
                    tgt_key_padding_mask: Optional[Tensor] = None,
                    memory_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None,
                    query_pos: Optional[Tensor] = None):
        tgt2 = self.norm1(tgt)
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt2, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.norm2(tgt)
        q = k = self.with_pos_embed(tgt2, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt2, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
    
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.norm3(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt2))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(tgt, memory, tgt_mask, memory_mask,
                                    tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
        return self.forward_post(tgt, memory, tgt_mask, memory_mask,
                                 tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)

In [37]:
class DetrClipFuserV2(nn.Module):
    def __init__(self, d_detr, d_clip, text_encoded, nhead=8,
                 num_layers=6, dim_feedforward=2048, dropout=0.1,
                 activation='relu', normalize_before=False,
                 return_intermediate=False) -> None:
        super().__init__()
        self.d_model = d_clip
        self.text_encoded = text_encoded
        self.linear_projection = nn.Linear(d_detr, d_clip)
        decoder_layer = TransformerDecoderLayer(self.d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        decoder_norm = nn.LayerNorm(self.d_model)
        self.decoder = TransformerDecoder(decoder_layer, num_layers, decoder_norm, return_intermediate)
    
    def forward(self, clip_img_feature, detr_proposals):
        detr_projected = self.linear_projection(detr_proposals)
        print(detr_projected.shape)
        out = self.decoder(tgt=detr_projected, memory=clip_img_feature)
        out = out.transpose(0, 1)
        logits = out @ self.text_encoded.T
        return logits

In [38]:
detr_features = torch.rand(100, 8, 256)
clip_features = torch.rand(1, 8, 512)
texts_encoded = torch.rand(80, 512)

detr_clip_fuser = DetrClipFuserV2(256, 512, texts_encoded, nhead=8)
out = detr_clip_fuser(clip_features, detr_features)
out.size()

torch.Size([100, 8, 512])


UnboundLocalError: cannot access local variable 'tgt2' where it is not associated with a value